In [1]:
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import glob
import time
import os
from scipy.stats import shapiro, mannwhitneyu

In [2]:
FORM = "%H:%M:%S.%f"
DATASET = {"b": "wiki-talk", "c": "kgs", "d": "datagen7.5"}
TESTS = ["tenantb-pagerank", "tenantb-sp", "tenantc-pr", "tenantc-sp", "tenantb-tc", "tenantc-tc", "tenantc-lpa", "tenantb-lpa"]

BASE = "C:/Users/hzwaan/Documents/serverlessgraphs/timestamps/multi/raw"

In [3]:
runs = glob.glob(f"{BASE}/*.csv")
timings = glob.glob(f"{BASE}/*.txt")

In [4]:
runnings_df = None  
timings_df = None

In [5]:
runnings_df = None  
for i, run in enumerate(runs):
    run_df = pd.read_csv(run)
    csv_path = run.split("\\",1)[1]
    identifier = csv_path.split('.')[0]

    dataset, algorithm = identifier.split("-")[:2]
    dataset = DATASET[dataset[-1]]
    algorithm = algorithm if algorithm != "pagerank" else "pr"

    run_number = int(identifier.split("-")[2])
    test_id = f"{identifier.split('-')[5]}-{run_number if run_number % 2 == 0 else run_number - 1}{'a' if run_number % 2 == 0 else 'b'}"

    if len(run_df) != 6:
        continue

    date = run_df.iloc[2]["Date"]
    creation = datetime.strptime(run_df.iloc[1]["Time"], FORM) - datetime.strptime(run_df.iloc[0]["Time"], FORM)
    if run_df.iloc[3]["Date"] != run_df.iloc[2]["Date"]:
        execution = (datetime.strptime(run_df.iloc[3]["Time"], FORM) + timedelta(days=1)) - datetime.strptime(run_df.iloc[2]["Time"], FORM)
    else:
        execution = datetime.strptime(run_df.iloc[3]["Time"], FORM) - datetime.strptime(run_df.iloc[2]["Time"], FORM)
    deletion = datetime.strptime(run_df.iloc[5]["Time"], FORM) - datetime.strptime(run_df.iloc[4]["Time"], FORM)

    creation_sec = int(creation.total_seconds())
    execution_sec = int(execution.total_seconds())
    deletion_sec = int(deletion.total_seconds())
    total_sec = creation_sec + execution_sec + deletion_sec

    begin_tp = run_df.iloc[2]["Time"]
    end_tp = run_df.iloc[3]["Time"]
    h,m,s = end_tp.split(":")
    s,ms = s.split(".")
    s = f"{str(int(s) + 5)}.{ms}" if int(s) + 5 <= 60 else "60.999999"
    if s == "60.999999":
        m = str(int(m) + 1)
    end_tp = f"{h}:{m}:{s}"

    result = pd.DataFrame([creation, execution, deletion, creation_sec, execution_sec, deletion_sec, total_sec, identifier, date, begin_tp, end_tp, test_id])
    result.index = ["creation", "execution", "deletion", "creation_sec", "execution_sec", "deletion_sec", "total_sec", "identifier", "date", "begin_tp", "end_tp", "test_id"]
    result = result.transpose()
    result["creation"] = result["creation"].astype(str).replace("0 days ", "", regex=True)
    result["execution"] = result["execution"].astype(str).replace("0 days ", "", regex=True)
    result["deletion"] = result["deletion"].astype(str).replace("0 days ", "", regex=True)
    result["dataset"] = dataset
    result["algorithm"] = algorithm
    if runnings_df is None:
        runnings_df = result
    else:
        runnings_df = runnings_df._append(result, ignore_index=True)

In [6]:
def date_format(time_stamp, day_offset = 0):
    if "day" in time_stamp:
        d, t = time_stamp.split("days")
        return datetime.strptime(t.strip(), FORM) + timedelta(days=(int(d) + day_offset))
    else:
        return datetime.strptime(time_stamp, FORM)

In [7]:
timings_df = None
for j, timing in enumerate(timings):
    execute = pd.read_csv(timing, sep=" ", header=None)
    execute.columns = ["Date", "Time"]
    execute.insert(0, "Action", ["start", "dataframe", "finished"])
    execute["Time"] = execute["Time"] + pd.Timedelta(hours=2)
    execute["Time"] = execute["Time"].astype(str).replace("0 days ", "", regex=True)
    identifier = timing.split("\\",1)[1]

    date = execute.iloc[1]["Date"]
    tp_dataframe_creation = date_format(execute.iloc[1]["Time"]) - date_format(execute.iloc[0]["Time"])

    if execute.iloc[2]["Date"] != execute.iloc[1]["Date"]:
        tp_execution = date_format(execute.iloc[2]["Time"], day_offset=1) - date_format(execute.iloc[1]["Time"])
    else:
        tp_execution = date_format(execute.iloc[2]["Time"]) - date_format(execute.iloc[1]["Time"])
        
    if "-" in str(tp_execution):
        tp_execution = tp_execution + timedelta(days=int(str(tp_execution).split("day")[0].replace("-", "")))


    tp_dataframe_sec = int(tp_dataframe_creation.total_seconds())
    tp_execution_sec = int(tp_execution.total_seconds())

    begin = execute.iloc[0]["Time"]
    end = execute.iloc[2]["Time"]

    result = pd.DataFrame([tp_dataframe_creation, tp_execution, tp_dataframe_sec, tp_execution_sec, date, begin, end, identifier])
    result.index = ["tp_dataframe", "tp_execution", "tp_dataframe_sec", "tp_execution_sec", "date", "begin", "end", "tp_identifier"]
    result = result.transpose()
    result["tp_dataframe"] = result["tp_dataframe"].astype(str).replace("0 days ", "", regex=True)
    result["tp_execution"] = result["tp_execution"].astype(str).replace("0 days ", "", regex=True)
    if timings_df is None:
        timings_df = result
    else:
        timings_df = timings_df._append(result, ignore_index=True)

In [8]:
total_results, test_ids = [], []
for time_row in timings_df.itertuples():
    _, algorithm, runinfo, date = time_row.tp_identifier[:-4].split("-",3)
    dataset = DATASET[runinfo[0]]
    runid = runinfo[1:]

    for run_row in runnings_df.itertuples():
        run_id, run_date = run_row.identifier.split("-",3)[2:]
        
        if runid == run_id and run_row.algorithm == algorithm and run_row.dataset == dataset and date == run_date:
            test_ids.append(run_row.test_id)
            total_results.append(run_row + time_row + (runid, run_id))
            break

resulting_df = pd.DataFrame(data=list(total_results), columns=
                            ["run_id", "creation", "execution", "deletion", "creation_sec",
                              "execution_sec", "deletion_sec", "total_sec", "identifier",
                              "date", "begin_tp", "end_tp", "test_id", "dataset", "algorithm", "timing_id", "tp_dataframe", 
                              "tp_execution", "tp_dataframe_sec", "tp_execution_sec", "date", 
                              "begin", "end", "tp_identifier", "A", "B"], index=test_ids)

In [9]:
resulting_df

run_id         creation        execution         deletion  \
21-0a        0  00:00:06.732954  00:02:50.611758  00:00:40.542498   
21-0b        2  00:00:06.478442  00:02:48.019097  00:00:39.046431   
21-10a       4  00:00:06.227217  00:02:53.457504  00:00:39.335801   
21-10b       5  00:00:06.252877  00:02:52.723097  00:00:42.070085   
21-12a       6  00:00:06.202211  00:03:20.006519  00:00:39.169061   
...        ...              ...              ...              ...   
22-58b      78  00:00:06.273334  00:07:12.994827  00:00:40.320064   
22-60a      79  00:00:06.216961  00:07:07.491083  00:00:39.943330   
22-60b      80  00:00:06.235570  00:07:21.512342  00:00:39.756474   
22-62a      81  00:00:06.233816  00:07:04.146157  00:00:45.765492   
22-62b      82  00:00:06.172536  00:07:13.158967  00:00:39.818959   

        creation_sec  execution_sec  deletion_sec  total_sec  \
21-0a              6            170            40        216   
21-0b              6            168            39        213   
21-10a             6            173            39        218   
21-10b             6            172            42        220   
21-12a             6            200            39        245   
...              ...            ...           ...        ...   
22-58b             6            432            40        478   
22-60a             6            427            39        472   
22-60b             6            441            39        486   
22-62a             6            424            45        475   
22-62b             6            433            39        478   

                            identifier        date  ...     tp_dataframe  \
21-0a    tenantb-pagerank-0-2023-07-21  2023-07-21  ...  00:01:47.332045   
21-0b    tenantb-pagerank-1-2023-07-21  2023-07-21  ...  00:01:47.022932   
21-10a  tenantb-pagerank-10-2023-07-21  2023-07-21  ...  00:01:47.299266   
21-10b  tenantb-pagerank-11-2023-07-21  2023-07-21  ...  00:01:46.703616   
21-12a  tenantb-pagerank-12-2023-07-21  2023-07-21  ...  00:01:47.210427   
...                                ...         ...  ...              ...   
22-58b        tenantc-sp-59-2023-07-22  2023-07-22  ...  00:06:13.971888   
22-60a        tenantc-sp-60-2023-07-22  2023-07-22  ...  00:06:10.612450   
22-60b        tenantc-sp-61-2023-07-22  2023-07-22  ...  00:06:15.955694   
22-62a        tenantc-sp-62-2023-07-22  2023-07-22  ...  00:06:12.374532   
22-62b        tenantc-sp-63-2023-07-22  2023-07-22  ...  00:06:16.990379   

           tp_execution tp_dataframe_sec tp_execution_sec        date  \
21-0a   00:00:54.710384              107               54  2023-07-21   
21-0b   00:00:53.900277              107               53  2023-07-21   
21-10a  00:00:58.767281              107               58  2023-07-21   
21-10b  00:00:58.868851              106               58  2023-07-21   
21-12a  00:01:05.140282              107               65  2023-07-21   
...                 ...              ...              ...         ...   
22-58b  00:00:51.463761              373               51  2023-07-22   
22-60a  00:00:49.135902              370               49  2023-07-22   
22-60b  00:00:47.504275              375               47  2023-07-22   
22-62a  00:00:43.072341              372               43  2023-07-22   
22-62b  00:00:48.280266              376               48  2023-07-22   

                  begin              end                  tp_identifier   A  \
21-0a   14:22:24.509672  14:25:06.552101   timings-pr-b0-2023-07-21.txt   0   
21-0b   14:22:33.602764  14:25:14.525973   timings-pr-b1-2023-07-21.txt   1   
21-10a  15:00:57.523039  15:03:43.589586  timings-pr-b10-2023-07-21.txt  10   
21-10b  15:01:07.059876  15:03:52.632343  timings-pr-b11-2023-07-21.txt  11   
21-12a  15:05:06.245851  15:07:58.596560  timings-pr-b12-2023-07-21.txt  12   
...                 ...              ...                            ...  ..   
22-58b  15:40:30.214786  15:47:35.650435  timings-sp-c59-2023-07-22

In [10]:
tp_start_info, tp_start_sec_info = [],[]
for _ in zip(resulting_df["begin_tp"], resulting_df["begin"]):
    tp_start = date_format(_[1]) - date_format(_[0])
    if "-1 day," in str(tp_start):
        tp_start = date_format(_[0]) - date_format(_[1])
    elif "day" in str(tp_start):
        tp_start = tp_start + timedelta(days=-int(str(tp_start).split("day")[0]))
    tp_start_sec = int(tp_start.total_seconds())
    tp_start_info.append(tp_start)
    tp_start_sec_info.append(tp_start_sec)
resulting_df["tp_start"] = tp_start_info
resulting_df["tp_start_sec"] = tp_start_sec_info
resulting_df["tp_start"] = resulting_df["tp_start"].astype(str).replace("0 days ", "", regex=True)

In [11]:
resulting_df["execution_remainder"] = resulting_df["execution_sec"] - resulting_df["tp_start_sec"] - resulting_df["tp_dataframe_sec"] - resulting_df["tp_execution_sec"]
resulting_df['execution_remainder'] = resulting_df['execution_remainder'].where(resulting_df['execution_remainder'] >= 0, 0)

In [12]:
resulting_df.groupby(["algorithm", "dataset"]).describe()

run_id                                                   \
                     count   mean       std   min    25%   50%    75%   max   
algorithm dataset                                                             
pr        kgs         20.0  51.50  5.916080  42.0  46.75  51.5  56.25  61.0   
          wiki-talk   20.0  11.35  6.175290   0.0   6.75  11.5  16.25  21.0   
sp        kgs         20.0  72.40  6.081897  62.0  67.75  72.5  77.25  82.0   
          wiki-talk   20.0  31.50  5.916080  22.0  26.75  31.5  36.25  41.0   

                    creation_sec       ... tp_start_sec        \
                           count mean  ...          75%   max   
algorithm dataset                      ...                      
pr        kgs               20.0  6.0  ...        17.00  69.0   
          wiki-talk         20.0  6.0  ...        21.25  54.0   
sp        kgs               20.0  6.0  ...        12.50  18.0   
          wiki-talk         20.0  6.0  ...        11.00  18.0   

                    execution_remainder                                   \
                                  count   mean        std  min  25%  50%   
algorithm dataset                                                          
pr        kgs                      20.0  11.35  20.332176  0.0  0.0  2.0   
          wiki-talk                20.0   2.25   2.593007  0.0  1.0  2.0   
sp        kgs                      20.0   1.45   0.887041  0.0  1.0  1.0   
          wiki-talk                20.0   1.80   1.436370  0.0  1.0  1.5   

                                 
                      75%   max  
algorithm dataset                
pr        kgs        9.25  63.0  
          wiki-talk  2.00  10.0  
sp        kgs        2.00   4.0  
          wiki-talk  2.00   7.0  

[4 rows x 80 columns]

In [13]:
resulting_df = resulting_df.drop_duplicates('run_id', keep=False)

In [14]:
resulting_df.groupby(["algorithm", "dataset"]).describe()

run_id                                                   \
                     count   mean       std   min    25%   50%    75%   max   
algorithm dataset                                                             
pr        kgs         20.0  51.50  5.916080  42.0  46.75  51.5  56.25  61.0   
          wiki-talk   20.0  11.35  6.175290   0.0   6.75  11.5  16.25  21.0   
sp        kgs         20.0  72.40  6.081897  62.0  67.75  72.5  77.25  82.0   
          wiki-talk   20.0  31.50  5.916080  22.0  26.75  31.5  36.25  41.0   

                    creation_sec       ... tp_start_sec        \
                           count mean  ...          75%   max   
algorithm dataset                      ...                      
pr        kgs               20.0  6.0  ...        17.00  69.0   
          wiki-talk         20.0  6.0  ...        21.25  54.0   
sp        kgs               20.0  6.0  ...        12.50  18.0   
          wiki-talk         20.0  6.0  ...        11.00  18.0   

                    execution_remainder                                   \
                                  count   mean        std  min  25%  50%   
algorithm dataset                                                          
pr        kgs                      20.0  11.35  20.332176  0.0  0.0  2.0   
          wiki-talk                20.0   2.25   2.593007  0.0  1.0  2.0   
sp        kgs                      20.0   1.45   0.887041  0.0  1.0  1.0   
          wiki-talk                20.0   1.80   1.436370  0.0  1.0  1.5   

                                 
                      75%   max  
algorithm dataset                
pr        kgs        9.25  63.0  
          wiki-talk  2.00  10.0  
sp        kgs        2.00   4.0  
          wiki-talk  2.00   7.0  

[4 rows x 80 columns]

In [15]:
# df_corrected = resulting_df[resulting_df["execution_remainder"] < 10] # remove problem executions
df_corrected = resulting_df

In [16]:
for _ in TESTS:
    __ = df_corrected[df_corrected.identifier.str.contains(_)]
    __.columns = [c.replace("_", "") for c in __.columns]
    __.to_csv(f"multi/clean/clean-{_}.csv")

In [17]:
for _ in TESTS:
    selected_combined = ["creation_sec", "tp_start_sec", "tp_dataframe_sec", "tp_execution_sec", "execution_remainder", "deletion_sec", "test_id"]
    combined = df_corrected[df_corrected.identifier.str.contains(_)][selected_combined]
    combined = combined.sort_values("test_id")
    # combined = combined.reset_index()
    combined = combined[selected_combined]

    if len(combined) == 0:
        continue

    data = "wiki-talk" if _.split("-")[0][-1:] == "b" else "kgs"
    algorithm = "pr" if _.split("-")[1] == "pagerank" else _.split("-")[1]

    ax = combined.plot(kind="bar", stacked=True, legend=False)
    # Add Title and Labels
    plt.title(f"Total execution time in seconds of {data} - {algorithm}")
    plt.xlabel("Run ID")
    plt.ylabel("Time (s)")
    plt.tight_layout()
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(handles[::-1], labels[::-1], loc=(1.015,0.6))
    plt.tight_layout()
    # for i, item in enumerate(reversed(ax.containers)):
    #     if i == 0:
    #         ax.bar_label(item, color="black", weight="bold")#, rotation=90, padding=2)
    plt.savefig(f"multi/bar/{_}-bar.png")
    plt.close()

    ax3 = df_corrected.boxplot(column="total_sec",rot=90)
    # # Add Title and Labels
    plt.title(f"Boxplot of {data} - {algorithm}")
    plt.xlabel("Part of total execution")
    plt.ylabel("Time (s)")
    plt.tight_layout()
    plt.savefig(f"multi/box/{_}-box.png")
    plt.close()

In [18]:
for _ in TESTS:
    df = df_corrected[df_corrected.identifier.str.contains(_)]

    if len(df) == 0:
        continue

    ax3 = df.boxplot(column="total_sec",rot=90)
    # # Add Title and Labels
    plt.title(f"Boxplot of {_}")
    plt.xlabel("Part of total execution")
    plt.ylabel("Time (s)")
    plt.tight_layout()
    plt.savefig(f"multi/box/{_}-box.png")
    plt.close()

In [19]:
for _ in TESTS:
    print(_)
    rows_df = df_corrected[df_corrected.identifier.str.contains(_)]

    if len(rows_df) == 0:
        continue
    print(shapiro(rows_df["total_sec"].tolist()))



    # try:
    #     _, pnorm = mannwhitneyu(res["1_default_prometheus"], res["2_final_settings"], method="asymptotic")
    #     print(pnorm)
    # except Exception as e:
    #     print(e)

    print(rows_df["total_sec"].describe())

tenantb-pagerank
ShapiroResult(statistic=0.8917247653007507, pvalue=0.028932448476552963)
count     20.000000
mean     231.700000
std       16.676804
min      212.000000
25%      216.750000
50%      226.000000
75%      245.250000
max      262.000000
Name: total_sec, dtype: float64
tenantb-sp
ShapiroResult(statistic=0.9345264434814453, pvalue=0.18863151967525482)
count     20.000000
mean     216.600000
std        4.967261
min      209.000000
25%      212.750000
50%      215.000000
75%      220.500000
max      225.000000
Name: total_sec, dtype: float64
tenantc-pr
ShapiroResult(statistic=0.9506898522377014, pvalue=0.3776830732822418)
count     20.000000
mean     548.950000
std       38.043569
min      492.000000
25%      521.250000
50%      549.000000
75%      566.250000
max      618.000000
Name: total_sec, dtype: float64
tenantc-sp
ShapiroResult(statistic=0.6599135994911194, pvalue=1.3152135579730384e-05)
count     20.000000
mean     481.600000
std       16.743891
min      455.000000
25%